# Farm Planning

This problem describes a European farmer's decision on how to divide their 500 acres of land among three crops: wheat, corn, and sugar beets. Each crop has specific characteristics, market prices, and requirements that impact the farmer's decision.

## Land Allocation Decision:

- The farmer wants to decide, during winter, how much of the 500 acres to dedicate to each crop.
- The farmer grows wheat and corn mainly to meet feed requirements for cattle, needing at least 200 tons of wheat and 240 tons of corn.
- If the farm doesn’t produce enough of these amounts, the farmer can buy wheat and corn from a wholesaler, but at a 40% higher price than selling prices due to added costs.

## Crop Sales and Prices:

- Wheat and corn can be sold if production exceeds the feed requirements.
- Based on past years, the farmer expects to sell wheat at \\$170/ton and corn at \\$150/ton. However, if they need to buy wheat or corn to meet feed needs, the prices increase to \\$238/ton for wheat and \\$210 perton for corn.
- Sugar beets are a profitable crop with a selling price of \\$36/ton, but there is a restriction: the farmer can sell only up to 6,000 tons at this price. Any sugar beet production beyond 6,000 tons can be sold only at a lower price of \$10/ton due to a quota imposed by the European Commission.

## Expected Yields:

- Based on past experience, the farmer has an average yield expectation per acre:
    - Wheat: 2.5 tons per acre.
    - Corn: 3 tons per acre.
    - Sugar beets: 20 tons per acre.
These yields help estimate how much land is needed for each crop to meet feed requirements and how much could be sold.

## Uncertainty in Crop Yields:

- Yields vary year by year depending on weather and other conditions.
- This problem assumes all crops will have similar changes in yield based on whether it’s a "good," "fair," or "bad" year:
    - In a good year, all crops yield 20% above the average.
    - In a fair year, all crops yield at the average level.
    - In a bad year, all crops yield 20% below the average.
- This approach simplifies uncertainty by assuming that environmental conditions impact all crops similarly (for example, good weather benefits all crops, and bad weather affects all negatively).

In [ ]:
using SDDP, JuMP, HiGHS, Gurobi

In [ ]:
struct ProbParams
    max_area        ::Float64
    crops           ::Vector{Symbol}
    planting_cost   ::Dict{Symbol, Float64}
    min_quantities  ::Dict{Symbol, Float64}
    quota_max       ::Dict{Symbol, Float64}
    sell_in_quota   ::Dict{Symbol, Float64}
    sell_no_quota   ::Dict{Symbol, Float64}
    buy_price       ::Dict{Symbol, Float64}
    mean_yield      ::Dict{Symbol, Float64}
    yield_multiplier::Dict{Symbol, Float64}
end

In [ ]:
function planning_t(sp::Model, stage::Int64; params::ProbParams)
    @variable(sp, area[c = params.crops] >= 0, SDDP.State, initial_value = 0)
    if stage == 1
        @constraint(sp, sum(sp[:area][c].out for c in params.crops) <= params.max_area)
        @stageobjective(sp, - sum(params.planting_cost[c] * sp[:area][c].out for c in params.crops))
    else
        @variables(sp, begin
            0 <= yield[c = params.crops]
            0 <= buy[c = params.crops]
            0 <= sell_in_quota[c = params.crops] <= params.quota_max[c]
            0 <= sell_no_quota[c = params.crops]
        end)
        @constraint(sp, [c = params.crops], 
            sp[:yield][c] + sp[:buy][c] - sp[:sell_in_quota][c] - sp[:sell_no_quota][c] >= params.min_quantities[c]
        )
        @constraint(sp, uncertainty[c = params.crops], 1.0 * sp[:area][c].in == sp[:yield][c])
        SDDP.parameterize(sp, collect(keys(config.yield_multiplier))) do ξ
            for c in params.crops
                JuMP.set_normalized_coefficient(
                    uncertainty[c], sp[:area][c].in, params.mean_yield[c] * params.yield_multiplier[ξ],
                )
            end
        end
        @stageobjective(sp,
            sum(
                params.sell_in_quota[c] * sp[:sell_in_quota][c] +
                params.sell_no_quota[c] * sp[:sell_no_quota][c] -
                params.buy_price[c] * sp[:buy][c] for c in params.crops
            )
        )
    end
end

In [ ]:
config = ProbParams(
    500.0,
    [:wheat, :corn, :sugar_beet],
    Dict(:wheat => 150.0, :corn => 230.0, :sugar_beet => 260.0),
    Dict(:wheat => 200.0, :corn => 240.0, :sugar_beet => 0.0),
    Dict(:wheat => Inf  , :corn => Inf  , :sugar_beet => 6_000.0),
    Dict(:wheat => 170.0, :corn => 150.0, :sugar_beet => 36.0),
    Dict(:wheat => 0.0  , :corn => 0.0  , :sugar_beet => 10.0),
    Dict(:wheat => 238.0, :corn => 210.0, :sugar_beet => 1_000.0),
    Dict(:wheat => 2.5  , :corn => 3.0  , :sugar_beet => 20.0),
    Dict(:good  => 1.2  , :fair => 1.0  , :bad        => 0.8)
)

In [ ]:
model = SDDP.LinearPolicyGraph(
    (sp, stage) -> planning_t(sp, stage; params = config);  # Closure with params
    stages = 2,
    sense  = :Max,
    upper_bound = 500_000.0,
    optimizer   = HiGHS.Optimizer,
)

In [ ]:
SDDP.train(model; iteration_limit = 40)

# Newsvendor

In [ ]:
using SDDP, HiGHS, Statistics, Test

In [ ]:
function get_uncertainty_support(; kwargs...)
    names  = tuple([first(kw) for kw in kwargs]...)
    values = tuple([last(kw) for kw in kwargs]...)
    output_type  = NamedTuple{names, Tuple{eltype.(values)...}}
    support = map(output_type, Base.product(values...))
    return support[:]
end

In [ ]:
## Expanding the get_uncertainty_support function

# Create a dictionary with parameters `supply` and `eps` each associated with ranges of values
kwargs = Dict(:supply => 0:0.05:0.5, :eps => [-0.25, -0.125, 0.125, 0.25])
# Extract the keys from `kwargs` as a tuple to use as names for fields in a NamedTuple
names  = tuple([key for key in keys(kwargs)]...)
# Extract the values from `kwargs` as a tuple to use as values for fields in a NamedTuple
vals   = tuple([value for value in values(kwargs)]...)
# Define the output type as a NamedTuple with field names `names` and field types matching `vals`
output_type = NamedTuple{names, Tuple{eltype.(vals)...}}
# Generate the cartesian product of all possible value combinations in `vals`
cartesian_product = collect(Base.product(vals...))
# Map each combination in the cartesian product to the `output_type` NamedTuple format
distribution = map(output_type, cartesian_product)

In [ ]:
function newsvendor_t(sp::Model, stage::Int64)
    @variables(sp, begin
        x >= 0, (SDDP.State, initial_value = 2)
        0 <= u <= 1
        w
    end)
    @constraint(sp, x.out == x.in - u + w)
    SDDP.add_objective_state(
        sp;
        initial_value = 1.5,
        lower_bound = 0.75,
        upper_bound = 2.25,
        lipschitz = 100.0,
    ) do incpt, ξ
        return incpt + ξ.eps
    end
    uncertainty = get_uncertainty_support(;
        supply = 0:0.05:0.5,
        eps    = [-0.25, -0.125, 0.125, 0.25],
    )
    SDDP.parameterize(sp, uncertainty) do ξ
        JuMP.fix(w, ξ.supply)
        price = SDDP.objective_state(sp)
        @stageobjective(sp, price * u)
    end
end

In [ ]:
model = SDDP.PolicyGraph(
    newsvendor_t,
    SDDP.LinearGraph(3);
    sense       = :Max,
    upper_bound = 50.0,
    optimizer   = HiGHS.Optimizer,
)

In [ ]:
SDDP.train(
    model;
    log_frequency = 10,
    time_limit    = 20.0,
    cut_type      = SDDP.MULTI_CUT,
)

In [ ]:
results = SDDP.simulate(model, 500)
objectives = [sum(s[:stage_objective] for s in simulation) for simulation in results]

In [ ]:
round(Statistics.mean(objectives); digits = 2)

In [ ]:
# Textbook implementation (sloppy)
function newsvendor_example(; cut_type)
    model = SDDP.PolicyGraph(
        SDDP.LinearGraph(3);
        sense = :Max,
        upper_bound = 50.0,
        optimizer = HiGHS.Optimizer,
    ) do subproblem, stage
        @variables(subproblem, begin
            x >= 0, (SDDP.State, initial_value = 2)
            0 <= u <= 1
            w
        end)
        @constraint(subproblem, x.out == x.in - u + w)
        SDDP.add_objective_state(
            subproblem;
            initial_value = 1.5,
            lower_bound = 0.75,
            upper_bound = 2.25,
            lipschitz = 100.0,
        ) do y, ω
            return y + ω.price_noise
        end
        noise_terms = get_uncertainty_support(;
            demand = 0:0.05:0.5,
            price_noise = [-0.25, -0.125, 0.125, 0.25],
        )
        SDDP.parameterize(subproblem, noise_terms) do ω
            JuMP.fix(w, ω.demand)
            price = SDDP.objective_state(subproblem)
            @stageobjective(subproblem, price * u)
        end
    end
    SDDP.train(
        model;
        log_frequency = 10,
        time_limit = 20.0,
        cut_type   = cut_type,
    )
    @test abs(SDDP.calculate_bound(model) - 4.04) <= 0.05
    results = SDDP.simulate(model, 500)
    objectives = [sum(s[:stage_objective] for s in simulation) for simulation in results]
    @test abs(round(Statistics.mean(objectives); digits = 2) - 4.04) <= 0.1
    return
end

newsvendor_example(; cut_type = SDDP.SINGLE_CUT)
newsvendor_example(; cut_type = SDDP.MULTI_CUT)